## Visualization of Shells

In [1]:
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
import itkwidgets as itkw
from vtkmodules.vtkIOXdmf2 import vtkXdmfReader
from vtkmodules.vtkCommonDataModel import vtkUnstructuredGrid
from vtkmodules.vtkFiltersGeometry import vtkGeometryFilter
from vtkmodules.vtkCommonExecutionModel import vtkStreamingDemandDrivenPipeline
import ipywidgets as ipw
import numpy as np
import pandas as pd
import h5py as hp
import numba

In [2]:
schedule = pd.read_csv('/home/amit/WorkSpace/UCLA/simulations/PhaseDiagram/Schedule.csv', sep='\t')

In [3]:
runw = ipw.IntSlider(min=0, max=2, step=1, value=0, description=r'Run', continuous_update=False)
filew = ipw.IntSlider(min=1, max=600, step=1, value=1, description=r'Simulation', continuous_update=False)
stepsw = ipw.IntSlider(min=0, max=999, step=1, value=0, description=r'Time Step', continuous_update=False)
check0w = ipw.Checkbox(value=True, description='Run0', disabled=False)
check1w = ipw.Checkbox(value=True, description='Run1', disabled=False)
check2w = ipw.Checkbox(value=True, description='Run2', disabled=False)
box = ipw.VBox([ipw.HBox([check0w, check1w, check2w]), ipw.HBox([runw, filew, stepsw])])

In [4]:
basedir = '/home/amit/WorkSpace/UCLA/simulations/PhaseDiagram/'
xdmffile = '{}/XDMFFiles/PhaseDiagram0/VTKFile-1.xdmf'.format(basedir)
rd = vtkXdmfReader()
rd.SetFileName(xdmffile)
rd.UpdateInformation()
info = rd.GetExecutive().GetOutputInformation(0)
gf = vtkGeometryFilter()
gf.SetInputConnection(rd.GetOutputPort())
gf.Update()
pd = gf.GetOutput()
pd.GetPointData().SetActiveNormals('PointNormals')
vtkview = itkw.view(geometries=pd)

def selectfile(change):
    run = runw.value
    fid = filew.value
    gamma = schedule.Gamma[fid - 1]
    temperature = schedule.Temperature[fid - 1]
    filename = '{}/XDMFFiles/PhaseDiagram{}/VTKFile-{}.xdmf'.format(basedir, run, fid)
    rd.SetFileName(filename)
    rd.UpdateInformation()
    info = rd.GetExecutive().GetOutputInformation(0)
    timesteps = info.Get(vtkStreamingDemandDrivenPipeline.TIME_STEPS())
    stepsw.max = timesteps[-1]
    stepsw.value = timesteps[-1]
    updateview(stepsw.value)
    
    
def updateview(data):
    time = data if isinstance(data, int) else data.new
    info.Set(vtkStreamingDemandDrivenPipeline.UPDATE_TIME_STEP(), time)
    rd.Modified()
    gf.Update()
    pd = gf.GetOutput()
    pd.GetPointData().SetActiveNormals('PointNormals')
    vtkview.geometries = pd

runw.observe(selectfile, names='value')
filew.observe(selectfile, names='value')
stepsw.observe(updateview, names='value')

In [5]:
def getasphericity(r, i):
    """
    Read asphericity from HDF5 file.
    """
    with hp.File('../RawData/PhaseDiagram{0}/VTKFile-{1}.h5'.format(r, i), 'r') as hfile:
        asph = hfile['Asphericity'][:]
    return asph


def get_volume_msd(rid, fid):
    filename = '{}/RawData/PhaseDiagram{}/DetailedOutput-{}.h5'.format(basedir, rid, fid)
    with hp.File(filename, 'r') as hfile:
        volume = hfile['Volume'][::2000]
        msd = hfile['MSD'][::2000]
    return volume, msd

In [6]:
plt.ioff()
plt.clf()
plt.style.use('ggplot')
plt.rc('text', usetex=True)

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, sharex=True)
fig.set_tight_layout(True)
fig.canvas.layout.max_width='750px'
fig.canvas.layout.height='700px'

ax0.set_ylabel('MSD')
ax1.set_ylabel('Volume')
ax2.set_ylabel('Asphericity')
ax0.set_ylim(0.0, 5.0)
ax1.set_ylim(20.0, 50.0)
ax2.set_ylim(-0.005, 0.05)

ax2.set_xlabel(r'Steps')
ax2.set_xlim(1, 2000000)

time = np.arange(0, 2000000, 2000)

text = r'$\gamma = {0:9.3f}$, $1/\beta = {1:6.4f}$'.format(
    schedule.Gamma[0], schedule.Temperature[0])
title = ax0.set_title(text)

msd0, = ax0.plot(time, get_volume_msd(0, 1)[1])
msd1, = ax0.plot(time, get_volume_msd(1, 1)[1])
msd2, = ax0.plot(time, get_volume_msd(2, 1)[1])
msdl  = [msd0, msd1, msd2]

vol0, = ax1.plot(time, get_volume_msd(0, 1)[0])
vol1, = ax1.plot(time, get_volume_msd(1, 1)[0])
vol2, = ax1.plot(time, get_volume_msd(2, 1)[0])
voll  = [vol0, vol1, vol2]

asph0, = ax2.plot(time, getasphericity(0, 1))
asph1, = ax2.plot(time, getasphericity(1, 1))
asph2, = ax2.plot(time, getasphericity(2, 1))
aspl = [asph0, asph1, asph2]

tl0 = ax0.axvline(x=0, ls='--', lw=0.5)
tl1 = ax1.axvline(x=0, ls='--', lw=0.5)
tl2 = ax2.axvline(x=0, ls='--', lw=0.5)
tvlines = [tl0, tl1, tl2]


def updateasphplot(change):
    fid = filew.value
    c0 = check0w.value
    c1 = check1w.value
    c2 = check2w.value
    
    gamma = schedule.Gamma[fid - 1]
    temperature = schedule.Temperature[fid - 1]
    
    text = r'$\gamma = {0:9.3f}$, $1/\beta = {1:6.4f}$'.format(gamma, temperature)
    title.set_text(text)
    
    for i, c in enumerate((c0, c1, c2)):
        if c:
            asph = getasphericity(i, fid)
            aspl[i].set_data(time, asph)
            vol, msd = get_volume_msd(i, fid)
            voll[i].set_data(time, vol)
            msdl[i].set_data(time, msd)
        else:
            msdl[i].set_data([], [])
            voll[i].set_data([], [])
            aspl[i].set_data([], [])
            
    tvalue = stepsw.value * 2000
    for tl in tvlines:
        tl.set_xdata(np.array([tvalue, tvalue]))
        
    fig.canvas.draw()
    fig.flush_events()


def updatetimelines(change):
    tvalue = stepsw.value * 2000
    for tl in tvlines:
        tl.set_xdata(np.array([tvalue, tvalue]))
    fig.canvas.draw()
    fig.flush_events()

    
runw.observe(updateasphplot, names='value')
filew.observe(updateasphplot, names='value')
stepsw.observe(updatetimelines, names='value')
check0w.observe(updateasphplot, names='value')
check1w.observe(updateasphplot, names='value')
check2w.observe(updateasphplot, names='value')

In [7]:
display(ipw.VBox([box, ipw.HBox([fig.canvas, vtkview])]))